# Test-04
Ejecutado en **Google Collaboratory**.

## Descripción
Se busca hacer uso de **Transfer Learning** con la red neuronal ya entrada **EfficientNetB0**. Para ello, fue necesario escalar las imágenes de entrada a un tamaño de 224x224, y se utilizan TPUs para optimizar el aprendizaje. Además, se guardan en memoria las imágenes con un nuevo tamaño para evitar el costo computacional durante entrenamiento.

# Fuentes

### Link: https://keras.io/api/layers/preprocessing_layers/image_preprocessing/resizing/
Se puede utilizar una capa que realiza una modificación en el tamaño de la imagen, particularmente como se busca aumentar el tamaño de las imágenes del CIFAR-100 a un tamaño aceptado por redes neuronales ya entrenadas como EfficientNet, se debe utilizar interpolación.

### Link: https://stackoverflow.com/questions/50760543/error-oom-when-allocating-tensor-with-shape/50764934
No hay suficiente memoria disponible en la GPU para poder guardar la información de un batch demasiado grande, por eso se usan batch_size pequeños.

### Link: https://stackoverflow.com/questions/47204116/shouldnt-model-trainable-false-freeze-weights-under-the-model
Las capas que no se reentrenan en Transfer Learning se tienen que 'congelar' antes de hacer una compilación del modelo.

# Configuración general

In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
!pip install cloud_tpu_client

In [4]:
from cloud_tpu_client import Client
c = Client()
c.configure_tpu_version(tf.__version__, restart_type="always")

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Running on TPU  ['10.40.164.34:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.40.164.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.40.164.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Cargando el conjunto de datos

In [6]:
import numpy as np

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
x_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_train.npy')
y_train_valid = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/y_train.npy')
x_test = np.load('drive/Shareddrives/Redes Neuronales BIO/CIFAR-100/x_test.npy')

# Separando el conjunto de datos

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, random_state=15, stratify=y_train_valid)

# Escalando las imágenes

In [11]:
from helper import resize_images

In [12]:
x_train_tf = resize_images(x_train, 224, 224)

In [13]:
x_train_tf.shape

TensorShape([40000, 224, 224, 3])

In [14]:
x_valid_tf = resize_images(x_valid, 224, 224)

In [15]:
x_valid_tf.shape

TensorShape([10000, 224, 224, 3])

In [16]:
x_test_tf = resize_images(x_test, 224, 224)

In [17]:
x_test_tf.shape

TensorShape([10000, 224, 224, 3])

# Modelo

In [18]:
from tensorflow.keras.applications import EfficientNetB0

In [19]:
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Activation, 
                                     BatchNormalization, Reshape, Dropout, InputLayer,
                                     Flatten)

In [20]:
from tensorflow.keras.layers.experimental.preprocessing import (Resizing,
                                                                RandomFlip,
                                                                RandomZoom,
                                                                RandomRotation,
                                                                RandomTranslation)

In [21]:
from tensorflow.keras.models import Sequential

In [22]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [23]:
from tensorflow.keras.optimizers import Adam

In [24]:
from tensorflow import keras

In [25]:
with strategy.scope():
  # Create the model
  model = Sequential()
  model.add(InputLayer(input_shape=(224, 224, 3)))
  model.add(EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3)))
  model.add(GlobalAveragePooling2D())
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(units=100))
  model.add(Activation('softmax'))

  # Compile the model
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=Adam(learning_rate=0.0001),
                metrics=['accuracy']
              )

# Summarize
model.summary()

16711680/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               128100    
_________________________________________________________________
activation (Activation)      (None, 100)               0         
Total params: 4,182,791
Trainable params: 4,138,208
Non-t

In [26]:
# Create the ModelCheckpoint callback to save the best model during training
mc_callback = ModelCheckpoint('model_1.hdf5',
                              monitor='val_accuracy',
                              save_best_only=True,
                              verbose=0,
                              mode='max'
                             )

# Create the EarlyStopping callback to stop when not improving during training
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

# Create the ReduceLROnPlateau callback to reduce the learning rate when not improving
lr_callback = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2, min_lr=0.001)

# Train the model
epochs = 50
batch_size = 256
model.fit(x_train_tf,
          y_train,
          validation_data=(x_valid_tf, y_valid), 
          callbacks=[mc_callback, es_callback, lr_callback],
          epochs=epochs,
          batch_size=batch_size
          )

Epoch 1/50
157/157 [==============================] - 101s 326ms/step - loss: 4.0313 - accuracy: 0.1756 - val_loss: 4.9670 - val_accuracy: 0.0173


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/50
157/157 [==============================] - 19s 118ms/step - loss: 1.9836 - accuracy: 0.4850 - val_loss: 3.5797 - val_accuracy: 0.2410
Epoch 3/50
157/157 [==============================] - 18s 118ms/step - loss: 1.4215 - accuracy: 0.6083 - val_loss: 1.1935 - val_accuracy: 0.6678
Epoch 4/50
157/157 [==============================] - 19s 118ms/step - loss: 1.1488 - accuracy: 0.6740 - val_loss: 0.7739 - val_accuracy: 0.7707
Epoch 5/50
157/157 [==============================] - 19s 118ms/step - loss: 0.9614 - accuracy: 0.7220 - val_loss: 0.6970 - val_accuracy: 0.7924
Epoch 6/50
157/157 [==============================] - 19s 119ms/step - loss: 0.8364 - accuracy: 0.7526 - val_loss: 0.6638 - val_accuracy: 0.8059
Epoch 7/50
157/157 [==============================] - 19s 118ms/step - loss: 0.7217 - accuracy: 0.7856 - val_loss: 0.6389 - val_accuracy: 0.8131
Epoch 8/50
157/157 [==============================] - 18s 117ms/step - loss: 0.6390 - accuracy: 0.8062 - val_loss: 0.6138 - val_ac

# Evaluación del modelo

In [27]:
from helper import generate_submission

In [28]:
# Load the model and show the final metrics
model = keras.models.load_model('model_1.hdf5')

# Train and validation metrics
_, train_acc = model.evaluate(x_train_tf, y_train, verbose=0)
_, valid_acc = model.evaluate(x_valid_tf, y_valid, verbose=0)

# Show result
print(f'[Accuracy] Train: {round(train_acc, 3)} Valid: {round(valid_acc, 3)}')

[Accuracy] Train: 0.997 Valid: 0.842


In [29]:
y_pred = model.predict(x_test_tf).argmax(axis=-1)

In [30]:
generate_submission(y_pred)